In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations.bbox_utils import denormalize_bbox, normalize_bbox

In [ ]:
class CustomCutout(DualTransform):
    """
    Custom Cutout augmentation with handling of bounding boxes 
    Note: (only supports square cutout regions)
    
    Author: Kaushal28
    Reference: https://arxiv.org/pdf/1708.04552.pdf
    """
    
    def __init__(
        self,
        fill_value=0,
        bbox_removal_threshold=0.50,
        min_cutout_size=192,
        max_cutout_size=512,
        always_apply=False,
        p=0.5
    ):
        """
        Class construstor
        
        :param fill_value: Value to be filled in cutout (default is 0 or black color)
        :param bbox_removal_threshold: Bboxes having content cut by cutout path more than this threshold will be removed
        :param min_cutout_size: minimum size of cutout (192 x 192)
        :param max_cutout_size: maximum size of cutout (512 x 512)
        """
        super(CustomCutout, self).__init__(always_apply, p)  # Initialize parent class
        self.fill_value = fill_value
        self.bbox_removal_threshold = bbox_removal_threshold
        self.min_cutout_size = min_cutout_size
        self.max_cutout_size = max_cutout_size
        
    def _get_cutout_position(self, img_height, img_width, cutout_size):
        """
        Randomly generates cutout position as a named tuple
        
        :param img_height: height of the original image
        :param img_width: width of the original image
        :param cutout_size: size of the cutout patch (square)
        :returns position of cutout patch as a named tuple
        """
        position = namedtuple('Point', 'x y')
        return position(
            np.random.randint(0, img_width - cutout_size + 1),
            np.random.randint(0, img_height - cutout_size + 1)
        )
        
    def _get_cutout(self, img_height, img_width):
        """
        Creates a cutout pacth with given fill value and determines the position in the original image
        
        :param img_height: height of the original image
        :param img_width: width of the original image
        :returns (cutout patch, cutout size, cutout position)
        """
        cutout_size = np.random.randint(self.min_cutout_size, self.max_cutout_size + 1)
        cutout_position = self._get_cutout_position(img_height, img_width, cutout_size)
        return np.full((cutout_size, cutout_size, 3), self.fill_value), cutout_size, cutout_position
        
    def apply(self, image, **params):
        """
        Applies the cutout augmentation on the given image
        
        :param image: The image to be augmented
        :returns augmented image
        """
        image = image.copy()  # Don't change the original image
        self.img_height, self.img_width, _ = image.shape
        cutout_arr, cutout_size, cutout_pos = self._get_cutout(self.img_height, self.img_width)
        
        # Set to instance variables to use this later
        self.image = image
        self.cutout_pos = cutout_pos
        self.cutout_size = cutout_size
        
        image[cutout_pos.y:cutout_pos.y+cutout_size, cutout_pos.x:cutout_size+cutout_pos.x, :] = cutout_arr
        return image
    
    def apply_to_bbox(self, bbox, **params):
        """
        Removes the bounding boxes which are covered by the applied cutout
        
        :param bbox: A single bounding box coordinates in pascal_voc format
        :returns transformed bbox's coordinates
        """

        # Denormalize the bbox coordinates
        bbox = denormalize_bbox(bbox, self.img_height, self.img_width)
        x_min, y_min, x_max, y_max = tuple(map(int, bbox))

        bbox_size = (x_max - x_min) * (y_max - y_min)  # width * height
        overlapping_size = np.sum(
            (self.image[y_min:y_max, x_min:x_max, 0] == self.fill_value) &
            (self.image[y_min:y_max, x_min:x_max, 1] == self.fill_value) &
            (self.image[y_min:y_max, x_min:x_max, 2] == self.fill_value)
        )

        # Remove the bbox if it has more than some threshold of content is inside the cutout patch
        if overlapping_size / bbox_size > self.bbox_removal_threshold:
            return normalize_bbox((0, 0, 0, 0), self.img_height, self.img_width)

        return normalize_bbox(bbox, self.img_height, self.img_width)

    def get_transform_init_args_names(self):
        """
        Fetches the parameter(s) of __init__ method
        :returns: tuple of parameter(s) of __init__ method
        """
        return ('fill_value', 'bbox_removal_threshold', 'min_cutout_size', 'max_cutout_size', 'always_apply', 'p')

def mixup(images, bboxes, areas, alpha=1.0):
    """
    Randomly mixes the given list if images with each other
    
    :param images: The images to be mixed up
    :param bboxes: The bounding boxes (labels)
    :param areas: The list of area of all the bboxes
    :param alpha: Required to generate image wieghts (lambda) using beta distribution. In this case we'll use alpha=1, which is same as uniform distribution
    """
    # Generate random indices to shuffle the images
    indices = torch.randperm(len(images))
    shuffled_images = images[indices]
    shuffled_bboxes = bboxes[indices]
    shuffled_areas = areas[indices]
    
    # Generate image weight (minimum 0.4 and maximum 0.6)
    lam = np.clip(np.random.beta(alpha, alpha), 0.4, 0.6)
    print(f'lambda: {lam}')
    
    # Weighted Mixup
    mixedup_images = lam*images + (1 - lam)*shuffled_images
    
    mixedup_bboxes, mixedup_areas = [], []
    for bbox, s_bbox, area, s_area in zip(bboxes, shuffled_bboxes, areas, shuffled_areas):
        mixedup_bboxes.append(bbox + s_bbox)
        mixedup_areas.append(area + s_area)
    
    return mixedup_images, mixedup_bboxes, mixedup_areas, indices.numpy()

In [ ]:
!ls ../input/nfl-impact-detection

In [ ]:
train_df = pd.read_csv('../input/nfl-impact-detection/train_labels.csv')

In [ ]:
train_df.groupby('playID').get_group(82).query('impact==1').iloc[:10]

In [ ]:
images_df = pd.read_csv('../input/nfl-impact-detection/image_labels.csv')
images_df.shape

In [ ]:
images_df['image'].unique()

In [ ]:
imgname = '57583_001841_Endzone_frame43.jpg'

img = cv2.imread(f'../input/nfl-impact-detection/images/{imgname}')

fig,ax = plt.subplots(figsize=(15,10))
ax.imshow(img)

sub_df = images_df.loc[images_df['image']==imgname][['left','width','top','height']]
for i,row in sub_df.iterrows():
    x,w,y,h = row.values
    rect = patches.Rectangle((x,y),w,h,
                             linewidth=1,
                             edgecolor='r',
                             facecolor='r',
                             alpha=0.25,
                            )
    ax.add_patch(rect)

plt.show()

In [ ]:
# pascal_voc, which is [x_min, y_min, x_max, y_max]
# COCO, which is [x_min, y_min, width, height]

In [ ]:
imgname = '57583_001841_Endzone_frame43.jpg'

In [ ]:
def check_aug(imgname,aug):
    image = cv2.imread(f'../input/nfl-impact-detection/images/{imgname}')
    boxes = images_df.loc[images_df['image']==imgname,['left','top','width','height']].values

    fig,axs = plt.subplots(figsize=(20,7.5),nrows=2,ncols=3)
    axs = axs.flatten()

    for i in range(6):
        if i==0:
            axs[i].imshow(image)
            sub_df = images_df.loc[images_df['image']==imgname][['left','width','top','height']]
            for j,row in sub_df.iterrows():
                x,w,y,h = row.values
                rect = patches.Rectangle((x,y),w,h,
                                         linewidth=1,
                                         edgecolor='r',
                                         facecolor='r',
                                         alpha=0.25,
                                        )
                axs[i].add_patch(rect)
        else:
            aug_result = aug(image=image, bboxes=boxes, labels=np.ones(boxes.shape))
            axs[i].imshow(aug_result['image'])
            for x,y,w,h in aug_result['bboxes']:
                rect = patches.Rectangle((x,y),w,h,
                                         linewidth=1,
                                         edgecolor='r',
                                         facecolor='r',
                                         alpha=0.25,
                                        )
                axs[i].add_patch(rect)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.VerticalFlip(1),    # Verticlly flip the image
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.Blur(blur_limit=(3,17),p=1.),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.CLAHE(clip_limit=6.0, tile_grid_size=(8, 8), p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.GaussNoise(var_limit=(10,10000),p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.Rotate(p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.HueSaturationValue(p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.RGBShift(p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.RandomBrightness(limit=(-0.25,0.25),p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.RandomContrast(limit=(-0.25,0.25),p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.RandomRain(p=1),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)

In [ ]:
aug = albumentations.Compose([
        #albumentations.Resize(512, 512),   # Resize the given 1024 x 1024 image to 512 * 512
        albumentations.VerticalFlip(p=0.5),    # Verticlly flip the image
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=(-0.25,0.25),p=0.5),
        albumentations.RandomContrast(limit=(-0.25,0.25),p=0.5),
        albumentations.RGBShift(p=0.5),
        albumentations.Blur(blur_limit=(3,17),p=0.5),
        albumentations.GaussNoise(var_limit=(10,1000),p=0.5),
    ], bbox_params={'format': 'coco','label_fields': ['labels']})
check_aug(imgname,aug)